# Diferenciación automática

En este notebook, veremos una metodología para calcular derivadas numéricamente de forma *exacta*, la **diferenciación automática** o **diferenciación algorítmica**

## Significado de la derivada

¿Para qué sirven las derivadas?

Es una medida de tasas de cambio

**[1]** (i) Supón que quieres calcular raíces de una función $f: \mathbb{R} \to \mathbb{R}$ como lo hemos hecho en un notebook anterior, y tienes una manera de calcular la derivada $f'$. Si calculas $f'(X)$ para un intervalo $X$, ¿qué te da? 

* El conjunto de todas las derivadas en el intervalo $X$ 

¿En qué te ayuda esto qué puedes concluir sobre las raíces? 

* Puedo implementar un método de Newton para intervalos $X$ para encontrar raices en el intervalo. Recordemos el método de Newton 

Sean $f:[a,b] \longrightarrow \mathbb{R}$ una función diferenciable a $[a,b]$ y $x_{0} \in [a,b]$ una condición inicial, definimos la función de iteración como $$x_{n+1} = x_{n} -  \dfrac{f(x_{n})}{f'(x_{n})} = \Phi(x_{n})$$ tal que

$\displaystyle\lim_{x_{n} \rightarrow \infty} \Phi(x_{n}) = x^{*}$ con $x^{*}$ la raíz de $f$

Ahora si extendemos esto a intevalo tenemos lo siguiente

Sean $f:X \longrightarrow \mathbb{R}$ una función diferenciable a $X$ y $x_{0} \in X$, punto medio del intervalos como condición inicial, definimos la función de iteración como $$x_{n+1} = x_{n} -  \dfrac{f(x_{n})}{f'(x_{n})}$$

(ii) ¿Se extiende esto a funciones $f:\mathbb{R^n} \to \mathbb{R}^n$?

* Sí

¿De qué manera?

* Con el método de Newton multidimensional

**[2]** Para una función $f:\mathbb{R} \to \mathbb{R}$, considera un intervalo $X$ donde queremos encontrar el rango $f(X)$. 

(i) Recuerda el teorema del valor medio ("mean value theorem"). Fija un $x_0 \in X$ y utiliza el teorema para escribir una expresión para $f(x)$ para $x \in X$, en términos de la derivada de $f$ en algún punto $\xi \in X$.

Por hipótesis $f\in \mathcal{C}^1$ por el teorema de Taylor, y acontado el residuo (de Lagrange), alrededor de $x_0$ fijo tenemos

$$f(x) = f(x_{0}) + f'(\xi)(x - x_{0})$$



(ii) No conocemos $\xi$. ¿Qué podemos hacer al respecto?

* Tomamos todo el intervalo $X$

Si lo haces, encontrarás otra expresión para el rango $f(X)$, que se conoce como el **mean value form** o **centred form**.

Utilizando la idea de acotar con intervalos, tenemos

$$f(X) = f(x_{0}) + f'(X)(X - x_{0})$$

(iii) Compara los resultados numéricos que obtienes de $f(X)$ con la extensión intervalar usual (lo que vimos en un notebook anterior) y esta versión nueva para distintas funciones. ¿Cuándo es mejor la nueva? 


(iv) Si tienes las dos formas de calcular el rango, ¿cómo puedes combinarlos? Impleméntalo.

* Para ello consideremos la función $f(x) = x^2$ en el intervalo $[-1,2]$ , que sabemos el rango es $f([-1,2]) = [0,4]$,  por otro lado usando la "nueva versión" con $x_0 = 1$

$$f([-1,2]) = f(1) + f'\left([-1,2] \right)([-1,2] - 1) = 1 + [-2,4]*[-2,1] = [-7,5] $$

Con $x_0 = 0$

$$f([-1,2]) = f(0) + f'\left([-1,2] \right)([-1,2] - 0) = [-2,4]*[-1,2] = [-4,8] $$

Notemos que ambos contienen el intervalo solución

Al calcular $f'(X)$ usamos la versión intervalar usual.

In [1]:
using Base.Test

In [2]:
struct Intervalo
    inf::Float64
    sup::Float64
    Intervalo(inf,sup) = inf > sup ? error("No es un intervalo valido") : new(inf,sup)
end

In [3]:
import Base: +, -,*, /, ^, exp, sqrt

function +(x::Intervalo, y::Intervalo)
    return Intervalo(x.inf + y.inf, x.sup + y.sup)
end

function +(x::Intervalo, c::Real) 
    # Aquí definimos la suma de un Intervalo a un Real
    return Intervalo(x.inf + c, x.sup + c)
end

function +(c::Real, x::Intervalo)
    # Aquí definimos la suma de un Real a un Intervalo
    return Intervalo(x.inf + c, x.sup + c)
end


function -(x::Intervalo)
    return Intervalo(- x.sup, - x.inf)
end

function -(x::Intervalo, y::Intervalo)
    y = -y
    return Intervalo(x.inf + y.inf, x.sup + y.sup)
end

function -(x::Intervalo, c::Real)
    return Intervalo(x.inf - c, x.sup - c)
end

function -(c::Real, x::Intervalo)
    return Intervalo(c-x.inf, c-x.sup)
end


function *(x::Intervalo, y::Intervalo)
    Intervalo(min(x.inf*y.inf, x.sup*y.inf, x.sup*y.sup, x.inf*y.sup), 
        max(x.inf*y.inf, x.sup*y.inf, x.sup*y.sup, x.inf*y.sup))
end

function *(x::Intervalo, c::Real)
    Intervalo(min(x.inf*c, x.sup*c), max(x.inf*c, x.sup*c))
    
    #return Intervalo(x.inf * c, x.sup * c)
end

function *(c::Real, x::Intervalo)
    Intervalo(min(x.inf*c, x.sup*c), max(x.inf*c, x.sup*c))
end

function /(x::Intervalo, y::Intervalo)
    if sign(y.inf) != sign(y.sup) && y.inf != 0.0 && y.sup != 0.0
        Intervalo(-Inf, Inf)
    elseif y.inf == 0.0
        Intervalo(min(x.inf/y.sup, x.sup/y.sup), Inf)
    elseif y.sup == 0.0
        Intervalo(-Inf, max(x.inf/y.inf, x.sup/y.inf))
    else
        Intervalo(min(x.inf/y.inf, x.inf/y.sup, x.sup/y.inf, x.sup/y.sup), 
            max(x.inf/y.inf, x.sup/y.inf, x.sup/y.sup, x.inf/y.sup))
    end
end

function /(x::Intervalo, c::Real)
    Intervalo(min(x.inf/c, x.sup/c), max(x.inf/c, x.sup/c))
    
    #return Intervalo(x.inf * c, x.sup * c)
end

function /(c::Real, x::Intervalo)
    if sign(x.inf) != sign(x.sup) && x.inf != 0.0 && x.sup != 0.0
        Intervalo(-Inf, Inf)
    elseif x.inf == 0.0
        Intervalo(c/x.sup, Inf)
    elseif x.sup == 0.0
        Intervalo(-Inf, c/x.inf)
    else
        Intervalo(min(c/x.inf, c/x.sup), max(c/x.inf, c/x.sup))
    end
end


function ^(x::Intervalo, c::Int64)
    if iseven(c)
        if x.inf < 0 && x.sup >0
            if abs(x.inf) < abs(x.sup)
                return Intervalo(0, x.sup^c)
            else
                return Intervalo(0, x.inf^c)
            end
        elseif x.inf < 0 && x.sup < 0
             return Intervalo(x.sup^c, x.inf^c)
        else
            return Intervalo(x.inf^c, x.sup^c)
        end
    else
        Intervalo(x.inf^c, x.sup^c)
    end
end

function exp(x::Intervalo)
    Intervalo(exp(x.inf),exp(x.sup))
end

function sqrt(x::Intervalo)
    if x.inf < 0 || x.sup < 0
        Intervalo(0,0)
    else
        Intervalo(sqrt(x.inf), sqrt(x.sup))
    end
end

sqrt (generic function with 11 methods)

In [4]:
## pruebas 1/x y 1/x-2

@show 1 / Intervalo(-1,1)
@show 1/(Intervalo(1,3) - 2)
@show 1/Intervalo(0,1)
@show 1/Intervalo(-1,0) 
@show 1/(Intervalo(-1,1))^2 ;

1 / Intervalo(-1, 1) = Intervalo(-Inf, Inf)
1 / (Intervalo(1, 3) - 2) = Intervalo(-Inf, Inf)
1 / Intervalo(0, 1) = Intervalo(1.0, Inf)
1 / Intervalo(-1, 0) = Intervalo(-Inf, -1.0)
1 / Intervalo(-1, 1) ^ 2 = Intervalo(1.0, Inf)


In [5]:
Intervalo(-1,2)^2

Intervalo(0.0, 4.0)

In [6]:
Intervalo(-1,2)^3

Intervalo(-1.0, 8.0)

In [7]:
g(x) = x^2
gp(x) = 2x

gp (generic function with 1 method)

In [8]:
typeof(g)

#g

In [9]:
function Fintervalo(f,fp, x::Intervalo , x0::Real)

    #x0 ∈ [x.inf, x.sup] ? nothing : error("x0 no esta en el intervalos")
    
    f(x0) + fp(x)*(x - x0)
    
end

Fintervalo (generic function with 1 method)

In [10]:
Fintervalo(g,gp,Intervalo(-1,2),1)

Intervalo(-7.0, 5.0)

## Diferenciación automática

De cálculo conocemos reglas para calcular derivadas (a diferencia, por ejemplo, de integrales indefinidas). Resulta que es posible *automatizar* estas reglas, para que la computadora las utilice para calcular derivadas de forma numéricamente exacta (a parte de errores de redondeo). Este conjunto de técnicas se llama la **diferenciación automática** o **diferenciación algorítmica**. En este notebook, veremos las bases de este método.

Nota: Este método *no* utiliza ni diferencias finitas, ni manipulación simbólica. Utiliza las reglas de derivadas provenientes del cálculo, pero de forma *numérica*. Especificamos el valor numérico de una variable, por ejemplo, `a = 3` en donde queremos evaluar la derivada de una función $f$, y ¡el método nos regresará el valor numérico de la derivada $f'(a)$!

**[3]** Supón que tienes dos funciones $f$ y $g$ de $\mathbb{R} \to \mathbb{R}$, cuyas derivadas conoces, y quieres calcular derivadas en un punto $a \in \mathbb{R}$ de combinaciones de estas funciones.

(i) Expande $f$ y $g$ en series de Taylor alrededor de $a$ en términos de la distancia $\epsilon$ desde $a$.
 
 Sea $\epsilon = x -a$

* $f(x) = f(a) + \dfrac{f'(a)(\epsilon)}{1!} + \dfrac{f''(a)(\epsilon)^2}{2!} + \cdots = \displaystyle \sum _{k = 0} ^{\infty} \dfrac{f^{(k)}(a) \epsilon ^k }{k!}$

* $g(x) = g(a) + \dfrac{g'(a)(\epsilon)}{1!} + \dfrac{g''(a)(\epsilon)^2}{2!} + \cdots = \displaystyle \sum _{k = 0} ^{\infty} \dfrac{g^{(k)}(a) \epsilon ^k }{k!} $


(ii) Encuentra series de Taylor para la suma $(f+g)$ y el producto $(f \cdot g)$ en la vecindad de $a$.

* $(f + g)(x) = \displaystyle \sum _{k = 0} ^{\infty}  \left[ f^{(k)}(a)  + g^{(k)}(a) \right]  \left(\dfrac{ \epsilon ^k}{k!}  \right) = \displaystyle \sum _{k =0 }^{\infty} \dfrac{(f + g)^{(k)}(a)\epsilon^{k}}{k!}  $

* $(f \cdot g)(x) = \left( \displaystyle \sum _{k = 0} ^{\infty} \dfrac{f^{(k)}(a) \epsilon ^k }{k!} \right)\left( \displaystyle \sum _{l = 0} ^{\infty} \dfrac{g^{(l)}(a) \epsilon ^l }{l!} \right) = \displaystyle \sum _{k =0 }^{\infty} \dfrac{(f \cdot g)^{(k)}(a)\epsilon^{k}}{k!}  $

(iii) Así, recobre las expresiones ya conocidas (de cálculo) para $(f+g)'(a)$ y $(f \cdot g)'(a)$,
en términos de los valores de las funciones y sus derivadas. Concluye cuál información necesitas de cada función para poder calcular derivadas en el punto $a$ de las combinaciones de las funciones.

* ($f + g)'(a) = f'(a) + g'(a)$, es decir, para suma necesitamos los coeficientes de orden epsilon

* $(f \cdot g)'(a) = f'(a)g(a) + f(a)g'(a)$, para esto necesitamos los coeficentes de orden menor o igual a epsilon de $f$ y $g$ 

Necesitamos que $f$ y $g$ sean $\mathcal{C}^{\infty}$ sin polos en $a$

**[4]** Ahora podemos convertir esto en un método numérico para calcular derivadas, como sigue.

(i) Define un tipo nuevo `Dual` que contiene la información necesaria de una función en el punto $a$. (Dejamos implícito el punto $a$; no lo representamos de forma explícita.) 

(ii) Define las operaciones aritméticas básicas sobre objetos de este tipo, siguiendo las reglas que desarrollamos en la pregunta (3).

(iii) ¿Cuál número `Dual` corresponde con la función identidad $\mathbb{1}: x \mapsto x$ en el punto $a$? Esto representará la "variable independiente" $x$.

* Es `Dual(a, 1)`

(iv) Verifica que si agarras un objeto `xx` de la forma encontrada en (iii) y lo enchufas en una función específica `f(x)` de Julia, arroja el valor de la función y de la derivada.

(v) Así, escribe una función que calcule de forma automática la derivada de una función en un punto dado.

In [11]:
import Base: ^

In [12]:
# (i)

struct Dual
    fa::Float64
    fpa::Float64
end

In [13]:
Dual(9,6)

Dual(9.0, 6.0)

In [14]:
function +(x::Dual, y::Dual)
    Dual(x.fa + y.fa, x.fpa + y.fpa)
end

function +(c::Real, y::Dual)
    Dual(c + y.fa, y.fpa)
end

function +(x::Dual, c::Real)
    Dual(x.fa + c, x.fpa )
end


function -(x::Dual, y::Dual)
    Dual(x.fa - y.fa, x.fpa - y.fpa)
end

function -(c::Real, y::Dual)
    Dual(c - y.fa, y.fpa)
end

function -(x::Dual, c::Real)
    Dual(x.fa - c, x.fpa )
end

function *(x::Dual, y::Dual)
    Dual(x.fa * y.fa, x.fpa * y.fa + x.fa * y.fpa )
end

function *(c::Real, y::Dual)
    Dual(c* y.fa,  c * y.fpa )
end

function *(x::Dual, c::Real)
    Dual(x.fa*c, x.fpa*c )
end

function ^(x::Dual, n::Int64)
    Dual(x.fa^n, n*x.fa^(n-1))
end

^ (generic function with 54 methods)

In [15]:
Dual(2,1)^3

Dual(8.0, 12.0)

In [16]:
Dual(9,6) + Dual(9,6) 

Dual(18.0, 12.0)

In [17]:
@testset "Pruebas de Dual" begin
    @test Dual(9,6) + Dual(9,6) == Dual(18,12)
    @test Dual(9,6) * Dual(9,6) == Dual(81,108)
    @test Dual(9,6) * Dual(1,0) == Dual(9,6)
    @test Dual(3,1) * Dual(3,1) == Dual(9,6)
end

Test Summary:   | Pass  Total
Pruebas de Dual |    4      4


Base.Test.DefaultTestSet("Pruebas de Dual", Any[], 4, false)

In [18]:
f(x) = x^3 - 2x

f (generic function with 1 method)

In [19]:
f(Dual(2,1))

Dual(4.0, 10.0)

In [20]:
function derivada(f,x0::Real)
    x = f(Dual(x0, 1))
    x.fpa
end

derivada (generic function with 1 method)

In [21]:
@test derivada(f,2) == Dual(4,10).fpa

Test Passed


**[5]** (i) Si tienes una función como `exp`, cuando la aplicas a un número `Dual` que representa a una función $f$ cerca del punto $a$, el resultado debe corresponder a la función $(\exp \circ f)$. Impleméntalo.

(ii) ¿Qué ocurre si derivas `exp(exp(x))` en un punto $a$? ¿Es correcto? ¿Qué se ha implementado (¡de forma automática!) aquí?

In [22]:
import Base: exp

function exp(x::Dual)
    Dual(exp(x.fa), exp(x.fa)*x.fpa)
end

exp (generic function with 12 methods)

In [23]:
exp(Dual(1,1))

Dual(2.718281828459045, 2.718281828459045)

In [24]:
g(x) = x^2 

g (generic function with 1 method)

In [25]:
g(Dual(3,1))

Dual(9.0, 6.0)

In [26]:
@test exp(g(Dual(3,1))) == exp(Dual(9,6))

Test Passed


(ii) ¿Qué ocurre si derivas `exp(exp(x))` en un punto $a$? ¿Es correcto? ¿Qué se ha implementado (¡de forma automática!) aquí?

In [27]:
exp(exp(Dual(1,1)))

Dual(15.154262241479262, 41.193555674716116)

In [28]:
expp(x) = exp(exp(x)) 
derivada(expp,1)

41.193555674716116

**[6]** Para una función $f: \mathbb{R}^n \to \mathbb{R}$, quisiéramos calcular la gradiente $\nabla f$. ¿Cómo podemos extender la diferenciación automática a este caso? Puedes restringir atención a $n=2$ para entender cómo funciona.

In [29]:
h(x,y) = x^2 + exp(y)

h2(x ,y) = x^2*y^2 

h3(x ,y) = x^2*y^2 + 2x + 3y

h4(x, y) = y^3 * exp(x)

h4 (generic function with 1 method)

In [30]:
h(0,1)

2.718281828459045

In [31]:
h2(Dual(2,1) , 1)

Dual(4.0, 4.0)

In [32]:
h2(2, Dual(1, 1))

Dual(4.0, 8.0)

In [33]:
h4(Dual(3,1), 2)

Dual(160.68429538550134, 160.68429538550134)

In [34]:
h4( 3, Dual(2,1))

Dual(160.68429538550134, 241.02644307825202)

In [35]:
function grad(f,x0,y0)
    xaux = f(Dual(x0, 1), y0)
    yaux = f(x0, Dual(y0, 1))

    Df = (xaux.fpa,yaux.fpa)

end

grad (generic function with 1 method)

In [36]:
grad(h, 2, 1 )

(4.0, 2.718281828459045)

In [37]:
grad(h2, 2, 1 )

(4.0, 8.0)

In [38]:
grad(h3, 2, 1 )

(6.0, 11.0)

In [39]:
grad(h4, 3, 2)

(160.68429538550134, 241.02644307825202)

In [40]:
12*exp(3)

241.02644307825202

**[7]** Ahora que hayamos entendido la idea de la diferenciación automática, podemos echar mano del paquete de Julia `ForwardDiff.jl`, donde hay una implementación muy buena.

Lee el manual del paquete para entender cómo calcular derivadas de funciones $\mathbb{R} \to \mathbb{R}$, gradientes de funciones $\mathbb{R}^n \to \mathbb{R}$, y jacobianos de funciones $\mathbb{R}^n \to \mathbb{R}^m$.

In [40]:
using ForwardDiff

Sean:

* $f:\mathbb{R} \longrightarrow \mathbb{R}$ tal que $f(x) = \sin(x^2)$ sabemos $f'(x) = 2x \cos (x^2)$ evaluando en $\sqrt{\pi}$ tenemos  $f'\left( \sqrt{\pi } \right) = 2 \sqrt{\pi} \cos (\pi) \approx -3.54490770181103$


* $g:\mathbb{R}^n \longrightarrow \mathbb{R}$ tal que $g(x,y,z) = \sin(x^2) + y\exp(z)$

In [43]:
f(x::Real) = sin(x^2);
g(x::Vector) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);

In [44]:
ForwardDiff.derivative(f, sqrt(pi))

-3.5449077018110318

In [45]:
x = rand(5) 
gg = x ->ForwardDiff.gradient(g, x)

(::#1) (generic function with 1 method)

In [46]:
gg(x)

5-element Array{Float64,1}:
 1.04183 
 1.29894 
 0.965813
 0.863309
 0.619499

In [47]:
ForwardDiff.hessian(g, x)

5×5 Array{Float64,2}:
 -0.168794  1.42623    0.209786   0.162717    0.156342 
  1.42623   0.293699   0.913246   0.708464    0.680335 
  0.209786  0.913246  -0.337995   0.104112    0.100055 
  0.162717  0.708464   0.104112  -0.506466    0.0776051
  0.156342  0.680335   0.100055   0.0776051  -0.716582 

**[8]** Ocupa diferenciación automática para calcular el "centered form" de una función $\mathbb{R} \to \mathbb{R}$.

In [48]:
?ForwardDiff.derivative

```
ForwardDiff.derivative(f, x::Real)
```

Return `df/dx` evaluated at `x`, assuming `f` is called as `f(x)`.

This method assumes that `isa(f(x), Union{Real,AbstractArray})`.

```
ForwardDiff.derivative(f!, y::AbstractArray, x::Real, cfg::DerivativeConfig = DerivativeConfig(f!, y, x))
```

Return `df!/dx` evaluated at `x`, assuming `f!` is called as `f!(y, x)` where the result is stored in `y`.


In [49]:
function centredform(f, X::Intervalo)
    x0 = abs(X.sup - X.inf)*0.5 + X.inf
    
    a = ForwardDiff.derivative(f, X.inf)
    b = ForwardDiff.derivative(f, X.sup)
    #@show x0
    if a >= b
        xp = Intervalo(b, a)
    else
        xp = Intervalo(a, b)
    end
    
    f(x0) + xp*(X - x0) 
end

centredform (generic function with 1 method)

In [50]:
ff(x) = x^2
II = Intervalo(-1, 2)

Intervalo(-1.0, 2.0)

In [51]:
centredform(ff, II)

Intervalo(-5.75, 6.25)

In [52]:
fff(x) = x^3  -3x^2 

fff (generic function with 1 method)

In [53]:
XX1 = Intervalo(-1,-.5)
XX2 = Intervalo(-1,1.5)
@show centredform(fff,XX1), fff(XX1)
@show centredform(fff,XX2), fff(XX2)

(centredform(fff, XX1), fff(XX1)) = (Intervalo(-4.359375, 0.140625), Intervalo(-4.0, -0.875))
(centredform(fff, XX2), fff(XX2)) = (Intervalo(-11.421875, 11.078125), Intervalo(-7.75, 3.375))


(Intervalo(-11.421875, 11.078125), Intervalo(-7.75, 3.375))

In [52]:
using Plots
gr()

equis1 = linspace(XX1.inf,XX1.sup, 200)
equis2 = linspace(XX2.inf,XX2.sup, 1000)

plot(equis2, fff.(equis2))
scatter!(equis1, fff.(equis1), markersize=0.3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
<polyline clip-path="url(#clip7602)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.0562,365.565 45.5764,363.635 46.0966,361.71 46.6168,359.793 47.137,357.881 47.6571,355.976 48.1773,354.078 48.6975,352.186 49.2177,350.3 49.7379,348.421 
 50.2581,346.548 50.7783,344.681 51.2985,342.821 51.8187,340.967 52.3388,339.12 52.859,337.279 53.3792,335.444 53.8994,333.616 54.4196,331.793 54.9398,329.978 
 55.46,328.168 55.9802,326.365 56.5004,324.568 57.0205,322.777 57.5407,320.993 58.0609,319.214 58.5811,317.443 59.1013,315.677 59.6215,313.917 60.1417,312.164 
 60.6619,310.417 61.1821,308.676 61.7022,306.942 62.2224,305.213 62.7426,303.491 63.2628,301.775 63.783,300.065 64.3032,298.362 64.8234,296.664 65.3436,294.973 
 65.8638,293.287 66.3839,291.608 66.9041,289.935 67.4243,288.268 67.9445,286.607 68.4647,284.953 68.9849,283.304 69.5051,281.662 70.0253,280.025 70.5455,278.395 
 71.0656,276.77 71.5858,275.152 72.106,273.54 72.6262,271.934 73.1464,270.334 73.6666,268.739 74.1868,267.151 74.707,265.569 75.2272,263.993 75.7473,262.423 
 76.2675,260.858 76.7877,259.3 77.3079,257.748 77.8281,256.202 78.3483,254.661 78.8685,253.127 79.3887,251.598 79.9089,250.076 80.429,248.559 80.9492,247.048 
 81.4694,245.543 81.9896,244.044 82.5098,242.551 83.03,241.064 83.5502,239.583 84.0704,238.107 84.5906,236.637 85.1107,235.174 85.6309,233.715 86.1511,232.263 
 86.6713,230.817 87.1915,229.376 87.7117,227.942 88.2319,226.513 88.7521,225.089 89.2723,223.672 89.7924,222.26 90.3126,220.854 90.8328,219.454 91.353,218.06 
 91.8732,216.671 92.3934,215.288 92.9136,213.911 93.4338,212.539 93.954,211.174 94.4741,209.814 94.9943,208.459 95.5145,207.11 96.0347,205.767 96.5549,204.43 
 97.0751,203.098 97.5953,201.772 98.1155,200.451 98.6357,199.137 99.1558,197.827 99.676,196.524 100.196,195.226 100.716,193.933 101.237,192.646 101.757,191.365 
 102.277,190.089 102.797,188.819 103.317,187.555 103.838,186.296 104.358,185.042 104.878,183.794 105.398,182.552 105.918,181.315 106.438,180.084 106.959,178.858 
 107.479,177.637 107.999,176.422 108.519,175.213 109.039,174.009 109.56,172.81 110.08,171.617 110.6,170.43 111.12,169.247 111.64,168.071 112.161,166.899 
 112.681,165.733 113.201,164.573 113.721,163.418 114.241,162.268 114.762,161.124 115.282,159.985 115.802,158.851 116.322,157.723 116.842,156.6 117.362,155.482 
 117.883,154.37 118.403,153.263 118.923,152.161 119.443,151.065 119.963,149.974 120.484,148.888 121.004,147.808 121.524,146.733 122.044,145.663 122.564,144.598 
 123.085,143.539 123.605,142.484 124.125,141.436 124.645,140.392 125.165,139.353 125.685,138.32 126.206,137.292 126.726,136.269 127.246,135.251 127.766,134.239 
 128.286,133.232 128.807,132.229 129.327,131.232 129.847,130.24 130.367,129.254 130.887,128.272 131.408,127.296 131.928,126.324 132.448,125.358 132.968,124.397 
 133.488,123.44 134.008,122.489 134.529,121.544 135.049,120.603 135.569,119.667 136.089,118.736 136.609,117.81 137.13,116.89 137.65,115.974 138.17,115.063 
 138.69,114.157 139.21,113.257 139.731,112.361 140.251,111.47 140.771,110.585 141.291,109.704 141.811,108.828 142.332,107.957 142.852,107.091 143.372,106.23 
 143.892,105.374 144.412,104.523 144.932,103.677 145.453,102.836 145.973,101.999 146.493,101.168 147.013,100.341 147.533,99.5193 148.054,98.7024 148.574,97.8903 
 149.094,97.0831 149.614,96.2807 150.134,95.4832 150.655,94.6904 151.175,93.9025 151.695,93.1194 152.215,92.3411 152.735,91.5676 153.255,90.7988 153.776,90.0348 
 154.296,89.2756 154.816,88.5212 155.336,87.7715 155.856,87.0265 156.377,86.2863 156.897,85.5508 157.417,84.82 157.937,84.0939 158.457,83.3726 158.978,82.6559 
 159.498,81.9439 160.018,81.2366 160.538,80.5339 161.058,79.8359 161.579,79.1426 162.099,78.4539 162.619,77.7698 163.139,77.0904 163.659,76.415

In [55]:
f4(x) = x + 1/x 

f41 = Intervalo(1,2)

@show centredform(f4,f41), f4(f41);

(centredform(f4, f41), f4(f41)) = (Intervalo(1.7916666666666665, 2.5416666666666665), Intervalo(1.5, 3.0))


In [54]:
g(x) = 1/x

gx=Intervalo(-1, 5.5)
@show centredform(g, gx)
@show g(gx)

centredform(g, gx) = Intervalo(-2.8055555555555554, 3.6944444444444446)
g(gx) = Intervalo(-Inf, Inf)


Intervalo(-Inf, Inf)

In [55]:
ForwardDiff.derivative(g,0.5)

-4.0

In [66]:
f5(x) = exp(sqrt(x))
f5X1 = Intervalo(0.1,8)
f5X2 = Intervalo(0,8)

@show centredform(f5, f5X1), centredform(f5, f5X2)
@show f5(f5X1), f5(f5X2)

(centredform(f5, f5X1), centredform(f5, f5X2)) = (Intervalo(-4.3321672216701534, 19.295584292804755), Intervalo(-Inf, Inf))
(f5(f5X1), f5(f5X2)) = (Intervalo(1.3719427019669197, 16.9188286785579), Intervalo(1.0, 16.9188286785579))


(Intervalo(1.3719427019669197, 16.9188286785579), Intervalo(1.0, 16.9188286785579))

In [65]:
equisf5 = linspace(f5X1.inf,f5X1.sup, 200)

plot(ylim=(0,18),yticks=0:1:18)
plot!(equisf5, f5.(equisf5))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 8 
 
 
 9 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 14 
 
 
 15 
 
 
 16 
 
 
 17 
 
 
 18 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,348.291 42.1477,346.628 44.9254,345.109 47.7031,343.685 50.4808,342.329 53.2584,341.024 56.0361,339.757 58.8138,338.521 61.5914,337.31 64.3691,336.12 
 67.1468,334.946 69.9244,333.786 72.7021,332.637 75.4798,331.498 78.2574,330.367 81.0351,329.242 83.8128,328.123 86.5904,327.008 89.3681,325.896 92.1458,324.787 
 94.9234,323.68 97.7011,322.574 100.479,321.469 103.256,320.365 106.034,319.26 108.812,318.154 111.589,317.048 114.367,315.941 117.145,314.832 119.922,313.721 
 122.7,312.608 125.478,311.493 128.255,310.375 131.033,309.254 133.811,308.131 136.588,307.004 139.366,305.874 142.144,304.741 144.921,303.604 147.699,302.463 
 150.477,301.318 153.254,300.169 156.032,299.016 158.81,297.859 161.587,296.697 164.365,295.531 167.143,294.361 169.92,293.185 172.698,292.005 175.476,290.82 
 178.253,289.63 181.031,288.435 183.809,287.235 186.586,286.029 189.364,284.819 192.142,283.603 194.919,282.382 197.697,281.155 200.475,279.922 203.252,278.684 
 206.03,277.441 208.808,276.191 211.585,274.936 214.363,273.675 217.141,272.408 219.918,271.135 222.696,269.856 225.474,268.571 228.251,267.28 231.029,265.982 
 233.807,264.679 236.584,263.369 239.362,262.053 242.14,260.73 244.918,259.401 247.695,258.066 250.473,256.724 253.251,255.375 256.028,254.02 258.806,252.658 
 261.584,251.29 264.361,249.915 267.139,248.533 269.917,247.144 272.694,245.748 275.472,244.346 278.25,242.936 281.027,241.52 283.805,240.096 286.583,238.666 
 289.36,237.228 292.138,235.783 294.916,234.331 297.693,232.872 300.471,231.406 303.249,229.932 306.026,228.451 308.804,226.963 311.582,225.467 314.359,223.964 
 317.137,222.453 319.915,220.935 322.692,219.409 325.47,217.875 328.248,216.334 331.025,214.786 333.803,213.229 336.581,211.665 339.358,210.093 342.136,208.514 
 344.914,206.926 347.691,205.331 350.469,203.728 353.247,202.116 356.024,200.497 358.802,198.87 361.58,197.235 364.357,195.591 367.135,193.94 369.913,192.28 
 372.69,190.612 375.468,188.936 378.246,187.252 381.023,185.559 383.801,183.858 386.579,182.149 389.356,180.431 392.134,178.705 394.912,176.971 397.689,175.227 
 400.467,173.476 403.245,171.716 406.022,169.947 408.8,168.169 411.578,166.383 414.355,164.588 417.133,162.785 419.911,160.972 422.688,159.151 425.466,157.321 
 428.244,155.482 431.021,153.635 433.799,151.778 436.577,149.912 439.354,148.037 442.132,146.153 444.91,144.26 447.687,142.358 450.465,140.447 453.243,138.527 
 456.02,136.597 458.798,134.658 461.576,132.71 464.353,130.753 467.131,128.786 469.909,126.81 472.686,124.824 475.464,122.829 478.242,120.824 481.019,118.81 
 483.797,116.786 486.575,114.753 489.352,112.71 492.13,110.657 494.908,108.594 497.685,106.522 500.463,104.44 503.241,102.348 506.018,100.247 508.796,98.1351 
 511.574,96.0136 514.351,93.8821 517.129,91.7406 519.907,89.5891 522.684,87.4276 525.462,85.2559 528.24,83.074 531.017,80.882 533.795,78.6798 536.573,76.4673 
 539.35,74.2445 542.128,72.0114 544.906,69.768 547.683,67.5141 550.461,65.2498 553.239,62.975 556.016,60.6898 558.794,58.3939 561.572,56.0875 564.349,53.7705 
 567.127,51.4429 569.905,49.1045 572.682,46.7554 575.46,44.3955 578.238,42.0249 581.015,39.6434 583.793,37.251 586.571,34.8477 589.348,32.4335 592.126,30.0082 
 
 "/>
 
 
 
 
 y1